In [1]:
import numpy as np
from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("data.csv")
dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset
Y = dataset
X = X[:, 0:2500]
Y = Y[:, 2500]

In [22]:
X_train = X[0:12000, :]
X_train = X_train/255.
X_test = X[12000:13201, :]
X_test = X_test/255.

In [23]:
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:12000, :]
Y_train = Y_train.T
Y_test = Y[12000:13201, :]
Y_test = Y_test.T

In [24]:
print("number of training examples = "+str(X_train.shape[0]))
print("number of test examples = "+ str(X_test.shape[0]))
print("X_train shape: "+ str(X_train.shape))
print("Y_train shape: "+ str(Y_train.shape))
print("X_test shape: "+ str(X_test.shape))
print("Y_test shape: "+ str(Y_test.shape))

number of training examples = 12000
number of test examples = 1200
X_train shape: (12000, 2500)
Y_train shape: (1, 12000)
X_test shape: (1200, 2500)
Y_test shape: (1, 1200)


In [25]:
image_x = 50
image_y = 50

In [26]:
train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], image_x, image_y, 1)
X_test = X_test.reshape(X_test.shape[0], image_x, image_y, 1)

In [27]:
print("X_train shape: "+ str(X_train.shape))
print("X_test shape: "+ str(X_test.shape))

X_train shape: (12000, 50, 50, 1)
X_test shape: (1200, 50, 50, 1)


In [28]:
print("train_y shape: " + str(train_y.shape))

train_y shape: (12000, 12)


In [29]:
def keras_model(image_x, image_y):
    num_of_classes = 12
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size=(5,5), input_shape = (image_x, image_y, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = 'same'))
    model.add(Conv2D(64, (5,5), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (5,5), strides = (5,5), padding = 'same'))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    filepath = "handEmo.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only = True, mode='max')
    callbacks_list = [checkpoint1]

    return model, callbacks_list

In [30]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=5, batch_size = 64, callbacks=callbacks_list)
scores = model.evaluate(X_test, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100- scores[1] *100))
print_summary(model)
model.save('handEmo.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 12000 samples, validate on 1200 samples
Epoch 1/5
12000/12000 [==============================] - 47s 4ms/step - loss: 0.1497 - acc: 0.9590 - val_loss: 0.0146 - val_acc: 0.9950

Epoch 00001: val_acc improved from -inf to 0.99500, saving model to handEmo.h5
Epoch 2/5
12000/12000 [==============================] - 39s 3ms/step - loss: 0.0082 - acc: 0.9972 - val_loss: 0.0069 - val_acc: 0.9975

Epoch 00002: val_acc improved from 0.99500 to 0.99750, saving model to handEmo.h5
Epoch 3/5
12000/12000 [==============================] - 38s 3ms/step - loss: 0.0034 - acc: 0.9990 - val_loss: 0.0027 - val_acc: 0.9983

Epoch 00003: val_acc improved from 0.99750 to 0.99833, saving model to handEmo.h5
Epoch 4/5
12000/12000 [==============================] - 35s 3ms/step - loss: 9.4